### 关于数据集 | About Dataset

https://www.kaggle.com/datasets/fivethirtyeight/the-ultimate-halloween-candy-power-ranking/data

#### 背景 | Context

What’s the best (or at least the most popular) Halloween candy?

#### 内容 | Content

在Candy.csv中包含了每种糖果的属性及其排名。对于二元变量，1表示“是”，0表示“否”。

- chocolate
  - 是否含有巧克力
- fruity
  - 是否是水果味的？
- caramel
  - 是否含有焦糖
- peanutalmondy (peanuts, peanut butter, almonds)
  - 花生杏仁类（是否含有花生、花生酱或杏仁？）
- nougat
  - 是否含有牛轧糖？
- crispedricewafer (crisped rice, wafers, cookie)
  - 脆米威化饼（是否含有脆米、威化饼或饼干成分？）
- hard
  - 是否坚硬？
- bar
  - 是否是棒状的？
- pluribus
  - 是否是一袋或一盒中的一个？
- sugarpercent: The percentile of sugar it falls under within the data set.
  - 含糖量百分比：在数据集中其含糖量所处的百分比区间。
- pricepercent: The unit price percentile compared to the rest of the set.
  - 价格百分比：与该组其他产品相比的单价百分位数。
- winpercent: The overall win percentage according to 269,000 matchups.
  - 胜率：根据26.9万投票得出的总胜率。

---